# Trabalho 2

In [173]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [174]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 768
largura = 1366
qtd_texturas = 50
window = glfw.create_window(largura, altura, "Trabalho 2", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [175]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [176]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [177]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [178]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [179]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [180]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [181]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [182]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [183]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



In [184]:
glEnable(GL_TEXTURE_2D)
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [185]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [186]:
modelo = load_model_from_file('terreno/pedra.obj')

print('Processando modelo piso.obj. Vertice inicial:',len(vertices_list))
ini_pedra = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo piso.obj. Vertice final:',len(vertices_list))
fim_pedra = len(vertices_list)

load_texture_from_file(1,'terreno/piso-2.jpg')


Processando modelo piso.obj. Vertice inicial: 0
Processando modelo piso.obj. Vertice final: 6


In [187]:
modelo = load_model_from_file('terreno/grama.obj')

print('Processando modelo grama.obj. Vertice inicial:',len(vertices_list))
ini_grama = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo grama.obj. Vertice final:',len(vertices_list))
fim_grama = len(vertices_list)

load_texture_from_file(4,'terreno/grama-1.jpg')


Processando modelo grama.obj. Vertice inicial: 6
Processando modelo grama.obj. Vertice final: 12


In [188]:
modelo = load_model_from_file('casa/casa.obj')

print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
ini_casa = len(vertices_list)
vertices_text = []
faces_visited = []
text_id = [2,5,6,7,8,9,10,11,12]
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))
fim_casa = len(vertices_list)
    
vertices_text.append(fim_casa)

load_texture_from_file(2,'casa/metal_dark_brown.jpg')
load_texture_from_file(5,'casa/metal_grey.jpg')
load_texture_from_file(6,'casa/wood_brown.jpg')
load_texture_from_file(7,'casa/plaster_light_brown.jpg')
load_texture_from_file(8,'casa/plaster_sand.jpg')
load_texture_from_file(9,'casa/plate_grey.jpg')
load_texture_from_file(10,'casa/wood_balls_brown.jpg')
load_texture_from_file(11,'casa/foundation_brown_brick.jpg')
load_texture_from_file(12,'casa/glass_window.jpg')


Processando modelo casa.obj. Vertice inicial: 12
metal_dark_brown vertice inicial:  12
metal_grey vertice inicial:  469578
wood_brown vertice inicial:  571176
plaster_light_brown vertice inicial:  580644
plaster_sand vertice inicial:  582819
plate_grey vertice inicial:  583779
wood_balls_brown vertice inicial:  583809
foundation_brown_brick vertice inicial:  584145
glass_window vertice inicial:  585381
Processando modelo casa.obj. Vertice final: 585477


In [189]:
modelo = load_model_from_file('sofa/sofa.obj')

print('Processando modelo sofa.obj. Vertice inicial:',len(vertices_list))
ini_sofa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo sofa.obj. Vertice final:',len(vertices_list))
fim_sofa = len(vertices_list)

load_texture_from_file(13,'sofa/sofa.jpg')

Processando modelo sofa.obj. Vertice inicial: 585477
Processando modelo sofa.obj. Vertice final: 590289


In [190]:
modelo = load_model_from_file('tv/tv.obj')

print('Processando modelo tv.obj. Vertice inicial:',len(vertices_list))
ini_tv = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo tv.obj. Vertice final:',len(vertices_list))
fim_tv = len(vertices_list)

load_texture_from_file(14,'tv/tv.png')

Processando modelo tv.obj. Vertice inicial: 590289
Processando modelo tv.obj. Vertice final: 1263057


In [191]:
modelo = load_model_from_file('arvore/arvore.obj')

print('Processando modelo arvore.obj. Vertice inicial:',len(vertices_list))
ini_arvore = len(vertices_list)
arvore_vertices_text = []
faces_visited = []
arvore_text_id = [15,16]
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        arvore_vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))
fim_arvore = len(vertices_list)
    
arvore_vertices_text.append(fim_arvore)

load_texture_from_file(15,'arvore/tronco.jpg')
load_texture_from_file(16,'arvore/folha.jpg')


Processando modelo arvore.obj. Vertice inicial: 1263057
Materiais.001 vertice inicial:  1263057
Materiais vertice inicial:  1350753
Processando modelo arvore.obj. Vertice final: 1408281


In [192]:
modelo = load_model_from_file('pato/pato.obj')

print('Processando modelo pato.obj. Vertice inicial:',len(vertices_list))
ini_pato = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo pato.obj. Vertice final:',len(vertices_list))
fim_pato = len(vertices_list)

load_texture_from_file(17,'pato/pato.jpg')

Processando modelo pato.obj. Vertice inicial: 1408281
Processando modelo pato.obj. Vertice final: 1460793


In [193]:
modelo = load_model_from_file('mesa/mesa.obj')

print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
ini_mesa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))
fim_mesa = len(vertices_list)

load_texture_from_file(18,'mesa/mesa.png')

Processando modelo mesa.obj. Vertice inicial: 1460793
Processando modelo mesa.obj. Vertice final: 1469547


In [194]:
modelo = load_model_from_file('ceu/ceu.obj')

print('Processando modelo ceu.obj. Vertice inicial:',len(vertices_list))
ini_ceu = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo ceu.obj. Vertice final:',len(vertices_list))
fim_ceu = len(vertices_list)

load_texture_from_file(19,'ceu/ceu.png')

Processando modelo ceu.obj. Vertice inicial: 1469547
Processando modelo ceu.obj. Vertice final: 1472427


In [195]:
modelo = load_model_from_file('carro/carro.obj')

print('Processando modelo carro.obj. Vertice inicial:',len(vertices_list))
ini_carro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carro.obj. Vertice final:',len(vertices_list))
fim_carro = len(vertices_list)

load_texture_from_file(20,'carro/carro.jpg')

Processando modelo carro.obj. Vertice inicial: 1472427
Processando modelo carro.obj. Vertice final: 1486629


In [196]:
modelo = load_model_from_file('cachorro/cachorro.obj')

print('Processando modelo cachorro.obj. Vertice inicial:',len(vertices_list))
ini_cachorro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cachorro.obj. Vertice final:',len(vertices_list))
fim_cachorro = len(vertices_list)

load_texture_from_file(21,'cachorro/cachorro.jpg')

Processando modelo cachorro.obj. Vertice inicial: 1486629
Processando modelo cachorro.obj. Vertice final: 1702533


In [197]:
modelo = load_model_from_file('homem/homem.obj')

print('Processando modelo homem.obj. Vertice inicial:',len(vertices_list))
ini_homem = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo homem.obj. Vertice final:',len(vertices_list))
fim_homem = len(vertices_list)

load_texture_from_file(22,'homem/homem.jpg')

Processando modelo homem.obj. Vertice inicial: 1702533
Processando modelo homem.obj. Vertice final: 1764159


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [198]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU

In [199]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [200]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [201]:
def desenha_arvore(_tx, _ty, _tz):
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = _tx; ty = _ty; tz = _tz;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 0)
    
    for i in range(0, len(arvore_text_id)):
        glBindTexture(GL_TEXTURE_2D, arvore_text_id[i])
        glDrawArrays(GL_TRIANGLES, arvore_vertices_text[i], arvore_vertices_text[i+1]-arvore_vertices_text[i])
    

In [202]:
def desenha_terreno():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = 0.0; ty = -1.01; tz = 0.0;
    
    # escala
    sx = 40.0; sy = 40.0; sz = 40.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 1)
    
    glDrawArrays(GL_TRIANGLES, ini_pedra, fim_pedra-ini_pedra)
    
    glBindTexture(GL_TEXTURE_2D, 4)
    
    glDrawArrays(GL_TRIANGLES, ini_grama, fim_grama-ini_grama)
    

In [203]:
def desenha_sofa():
    
    # rotacao
    angle = 270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 18.5; ty = -0.9; tz = -5.0;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 13)
    
    glDrawArrays(GL_TRIANGLES, ini_sofa, fim_sofa-ini_sofa)

In [204]:
def desenha_casa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -8.0; ty = -1.0; tz = 5.0;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(text_id)):
        glBindTexture(GL_TEXTURE_2D, text_id[i])
        glDrawArrays(GL_TRIANGLES, vertices_text[i], vertices_text[i+1]-vertices_text[i])
    

In [205]:
def desenha_tv():
    
    # rotacao
    angle = -270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -3.5; ty = 1.1; tz = -4.5;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 14)
    
    glDrawArrays(GL_TRIANGLES, ini_tv, fim_tv-ini_tv)

In [206]:
def desenha_pato():
    
    # rotacao
    angle = 0.0;
    rx = -5.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -20; ty = -1.0; tz = 9.0;
    
    # escala
    sx = 0.3; sy = 0.3; sz = 0.3;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 17)
    
    glDrawArrays(GL_TRIANGLES, ini_pato, fim_pato-ini_pato)

In [207]:
def desenha_mesa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -4.0; ty = -1.0; tz = -4.5;
    
    # escala
    sx = 6.0; sy = 6.0; sz = 6.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 18)
    
    glDrawArrays(GL_TRIANGLES, ini_mesa, fim_mesa-ini_mesa)

In [208]:
def desenha_ceu():
    
    # rotacao
    angle = 90.0;
    rx = 90.0; ry = 1.0; rz = 90.0;
    
    # translacao
    tx = 60.0; ty = -60.0; tz = -140.5;
    
    # escala
    sx = 30.0; sy = 30.0; sz = 30.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 19)
    
    glDrawArrays(GL_TRIANGLES, ini_ceu, fim_ceu-ini_ceu)

In [209]:
def desenha_carro():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -20; ty = -1.0; tz = -5.0;
    
    # escala
    sx = 4.0; sy = 4.0; sz = 4.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    glBindTexture(GL_TEXTURE_2D, 20)
    
    glDrawArrays(GL_TRIANGLES, ini_carro, fim_carro-ini_carro)

In [210]:
def desenha_cachorro():
    
    # rotacao
    angle = 0.0;
    rx = -5.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 25; ty = -1.0; tz = 9.0;
    
    # escala
    sx = 0.15; sy = 0.15; sz = 0.15;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 21)
    
    glDrawArrays(GL_TRIANGLES, ini_cachorro, fim_cachorro-ini_cachorro)

In [211]:
def desenha_homem():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 15.5; ty = -1.0; tz = -15.0;
    
    # escala
    sx = 0.4; sy = 0.4; sz = 0.4;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 22)
    
    glDrawArrays(GL_TRIANGLES, ini_homem, fim_homem-ini_homem)

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [212]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 2.5
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [213]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [214]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [215]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    desenha_arvore(30.0, -1.0, 30.0)
    desenha_arvore(-30.0, -1.0, 30.0)   
    desenha_terreno()
    desenha_casa()
    desenha_sofa()
    desenha_tv()
    desenha_pato()
    desenha_mesa()
    desenha_ceu()
    desenha_carro()
    desenha_cachorro()
    desenha_homem()
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    

    
    glfw.swap_buffers(window)

glfw.terminate()